In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.base import TransformerMixin
from skimage.io import imread, imread_collection_wrapper, concatenate_images

In [3]:
import sys
sys.path.append("../")
from config import DATASETS_PATH, TRAIN_PATH, VALIDATION_PATH
from helpers.utils import extract_label_values

# Load data

In [4]:
img_file_pattern = "*.tif"
train_imgs_path = os.path.join(TRAIN_PATH, img_file_pattern)
"""
it is necessary to create custom imread collection function which reads images with the 'imread' function
in order to obtain the raw values from the tif image. 
The default imread_collection function returns images that are uncorrectly scaled between 0 and 255
"""
imread_collection_custom = imread_collection_wrapper(imread)
train_imgs = imread_collection_custom(train_imgs_path, conserve_memory=True)

# Feature extraction

## spectral features

In [6]:
# TODO: move classes extracting features to separate python file
class BaseFeatureExtractor(TransformerMixin):
    def __init__(self):
        self.pixels_axis = (1, 2)
    
    def fit(self, imgs, y=None):
        raise NotImplementedError
    
    def transform(self, imgs, y=None):
        raise NotImplementedError

class SpectralFeatureExtractor(BaseFeatureExtractor):
    """
    extracts mean and standard deviation of every color channel in the image (RGB)
    and the brightness, where brightness is defined as the mean of all color channels

    Parameters
    ----------
    imgs : numpy.ndarray
           set of images, each with 4 channels (B, G, R, NIR)
    """
    def __init__(self):
        super().__init__()
    
    def fit(self, imgs, y=None):
        return self
    
    def transform(self, imgs, y=None):
        imgs = imgs[:, :, :, :3] # extract color channels
        rgb_means = np.mean(imgs, axis=self.pixels_axis)
        brightness = np.mean(rgb_means, axis=1)
        brightness = np.reshape(brightness, (-1, 1))
        rgb_sds = np.std(imgs, axis=self.pixels_axis)

        return np.concatenate((rgb_means, brightness, rgb_sds), axis=1)

class NDVIFeatureExtractor(BaseFeatureExtractor):
    """
    extracts normalized difference vegatation index from multispectral image

    Parameters
    ----------
    imgs : numpy.ndarray
           set of images, each with 4 channels (B, G, R, NIR)
    """
    def __init__(self):
        super().__init__()
    
    def fit(self, imgs, y=None):
        return self
    
    def transform(self, imgs, y=None):
        red = imgs[:, :, :, 2]
        nir = imgs[:, :, :, 3]

        ndvi = np.divide(nir-red, nir+red)
        
        ndvi_means = np.mean(ndvi, axis=self.pixels_axis)
        ndvi_sds = np.std(ndvi, axis=self.pixels_axis)
        ndvi_means = np.reshape(ndvi_means, (-1, 1))
        ndvi_sds = np.reshape(ndvi_sds, (-1, 1))
        return np.concatenate((ndvi_means, ndvi_sds), axis=1)
        

In [7]:
feature_extractor = FeatureUnion(transformer_list=[
    ("spectral", SpectralFeatureExtractor()),
    ("ndvi", NDVIFeatureExtractor())
])

In [42]:
def extract_features(imgs_collection, feature_extractor, batch_size=1000):
    """
    Extracts from imgs_collection the set of features specified in feature_extractor.
    Extracts the features in batches because it is unviable to load all images at once into memory
    
    Parameters
    ----------
    imgs_collection : skimage.ImageCollection
                      collection of images from which we want to extract the features
    feature_extractor: sklearn transformer
                       transformers that extract features from images
    batch_size: number of images to extract features from at each iteration
                a deafult value of 1000 loads approx. 0.5 GB into memory for this dataset
    
    Returns
    -------
    features: numpy.ndarray
              set of features extracted from the image, 
              with one row for each image and one column for each feature
    """
    
    n_images = len(imgs_collection)
    # get number of total features
    features_im0 = feature_extractor.fit_transform(concatenate_images(train_imgs[:1]))
    n_features = np.shape(features_im0)[1]
    # create array for features
    features = np.zeros([n_images, n_features])
    features[0, :] = features_im0
    for i in range(1, n_images, batch_size):
        imgs_batch = concatenate_images(train_imgs[i:i+batch_size])
        # casting to float64 is required to extract features! 
        # else 
        imgs_batch = imgs_batch.astype('float64', casting='safe')
        features_batch = feature_extractor.fit_transform(imgs_batch)
        features[i:i+batch_size, :] = features_batch
    return features

In [43]:
features = extract_features(train_imgs, feature_extractor)

# model

In [ ]:
classifier = RandomForestClassifier(n_estimators=500)

In [ ]:
train_labels = pd.read_csv(os.path.join(DATASETS_PATH, 'train_labels.csv'))
train_labels = extract_label_values(train_labels)
validation_labels = pd.read_csv(os.path.join(DATASETS_PATH, 'validation_labels.csv'))
validation_labels = extract_label_values(validation_labels)

In [ ]:
# validation